Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [60]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [61]:
df.dtypes

Burrito            object
Date               object
Yelp              float64
Google            float64
Chips              object
Cost              float64
Hunger            float64
Mass (g)          float64
Density (g/mL)    float64
Length            float64
Circum            float64
Volume            float64
Tortilla          float64
Temp              float64
Meat              float64
Fillings          float64
Meat:filling      float64
Uniformity        float64
Salsa             float64
Synergy           float64
Wrap              float64
Unreliable         object
NonSD              object
Beef               object
Pico               object
Guac               object
Cheese             object
Fries              object
Sour cream         object
Pork               object
Chicken            object
Shrimp             object
Fish               object
Rice               object
Beans              object
Lettuce            object
Tomato             object
Bell peper         object
Carrots     

In [0]:
df["year"] = pd.to_datetime(df["Date"]).apply(lambda x: x.year)

In [0]:
# handle NaNs
# number columns: NaN -> median
# object columns: looks like NaN means "no", switch to 0/1 representation

import numpy as np

number_cols = df.select_dtypes(include="number").columns
object_cols = df.select_dtypes(include="object").columns

df[number_cols] = df[number_cols].fillna(df[number_cols].median(skipna=True))

In [64]:
for col in object_cols:
    print(df[col].value_counts())

California     169
Other          156
Asada           43
Surf & Turf     28
Carnitas        25
Name: Burrito, dtype: int64
8/30/2016     29
8/27/2019      9
6/24/2016      9
5/6/2016       7
4/15/2016      7
              ..
1/10/2017      1
7/13/2017      1
11/14/2016     1
9/13/2016      1
12/15/2016     1
Name: Date, Length: 169, dtype: int64
x      21
X       3
No      1
Yes     1
Name: Chips, dtype: int64
x    33
Name: Unreliable, dtype: int64
x    5
X    2
Name: NonSD, dtype: int64
x    137
X     42
Name: Beef, dtype: int64
x    127
X     31
Name: Pico, dtype: int64
x    114
X     40
Name: Guac, dtype: int64
x    128
X     31
Name: Cheese, dtype: int64
x    102
X     25
Name: Fries, dtype: int64
x    67
X    25
Name: Sour cream, dtype: int64
x    36
X    15
Name: Pork, dtype: int64
x    20
X     1
Name: Chicken, dtype: int64
x    17
X     4
Name: Shrimp, dtype: int64
x    4
X    2
Name: Fish, dtype: int64
x    26
X    10
Name: Rice, dtype: int64
x    27
X     8
Name: Beans, dtype

In [0]:
df[object_cols] = df[object_cols].replace(
    {
        "x": 1,
        "X": 1,
        "Yes": 1,
        "No": 0,
        np.NaN: 0,
     }
)

In [66]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great,year
0,California,1/18/2016,3.5,4.2,0,6.49,3.0,540.0,0.658099,20.0,22.0,0.77,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,False,2016
1,California,1/24/2016,3.5,3.3,0,5.45,3.5,540.0,0.658099,20.0,22.0,0.77,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,False,2016
2,Carnitas,1/24/2016,4.0,4.2,0,4.85,1.5,540.0,0.658099,20.0,22.0,0.77,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,False,2016
3,Asada,1/24/2016,4.0,4.2,0,5.25,2.0,540.0,0.658099,20.0,22.0,0.77,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,False,2016
4,California,1/27/2016,4.0,3.8,1,6.59,4.0,540.0,0.658099,20.0,22.0,0.77,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,True,2016


In [67]:
df.isnull().sum()

Burrito             0
Date                0
Yelp                0
Google              0
Chips               0
Cost                0
Hunger              0
Mass (g)            0
Density (g/mL)      0
Length              0
Circum              0
Volume              0
Tortilla            0
Temp                0
Meat                0
Fillings            0
Meat:filling        0
Uniformity          0
Salsa               0
Synergy             0
Wrap                0
Unreliable          0
NonSD               0
Beef                0
Pico                0
Guac                0
Cheese              0
Fries               0
Sour cream          0
Pork                0
Chicken             0
Shrimp              0
Fish                0
Rice                0
Beans               0
Lettuce             0
Tomato              0
Bell peper          0
Carrots             0
Cabbage             0
Sauce               0
Salsa.1             0
Cilantro            0
Onion               0
Taquito             0
Pineapple 

In [0]:
df = df[df.columns.drop(["Queso"])]

In [69]:
from plotly import express as px

ratings_df = df.pivot_table(values=["Yelp", "Google"], index=["Burrito"]).reset_index(["Burrito"])

px.bar(ratings_df, x="Burrito", y="Google")

In [0]:
# Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
train = df[df["year"] <= 2016]
val = df[df["year"] == 2017]
test = df[df["year"] >= 2018]

In [0]:
# Begin with baselines for classification.
target = "Google"

y_train = train[target]
y_val = val[target]
y_test = test[target]

mode = y_train.mode()[0]
y_pred = [mode] * len(y_val)


In [72]:
from sklearn.metrics import accuracy_score, mean_absolute_error

mean_absolute_error(y_pred, y_val)

0.047058823529411764

In [77]:
# Use scikit-learn for logistic regression.
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs")

features = df.columns.drop(["Burrito", "Date", "Yelp", "Google"])

X_train = train[features]
X_val = val[features]
X_test = test[features]

ValueError: ignored

In [0]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()

X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)
X_test_imputed = imputer.transform(X_test)

In [76]:
model.fit(X_train_imputed, y_train)
model.predict(X_val_imputed)

ValueError: ignored